In [108]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 80)

In [109]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1VEpP7kLJjlLR9MyTgu2FyFnOJArh6U2b",
    'housing_data_ordy':"1jHl0STmNul4uPk9NeFrgtIuOnt8SqrgF" # adress to itaration 6 
}


housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
df = housing_data

# Pre-Processing Pipe

In [25]:
df

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,KitchenQual,FireplaceQu,MSSubClass,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,BsmtFinType2,HeatingQC,Electrical,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,No,GLQ,Gd,NaN,60,7,5,2003,2003,196.0,706,0,150,856,854,0,1710,1,0,2,1,1,8,2003.0,548,61,0,0,0,2,2008,1,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,RL,Feedr,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,Gd,ALQ,TA,TA,20,6,8,1976,1976,0.0,978,0,284,1262,0,0,1262,0,1,2,0,1,6,1976.0,460,0,0,0,0,5,2007,2,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,Mn,GLQ,Gd,TA,60,7,5,2001,2002,162.0,486,0,434,920,866,0,1786,1,0,2,1,1,6,2001.0,608,42,0,0,0,9,2008,3,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,RL,Norm,GasA,Pave,Y,BrkTil,TA,TA,TA,Gd,No,ALQ,Gd,Gd,70,7,5,1915,1970,0.0,216,0,540,961,756,0,1717,1,0,1,0,1,7,1998.0,642,35,272,0,0,2,2006,4,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,NaN,Unf,Gd,SBrkr,Typ,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,Av,GLQ,Gd,TA,60,8,5,2000,2000,350.0,655,0,490,1145,1053,0,2198,1,0,2,1,1,9,2000.0,836,84,0,0,0,12,2008,5,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,RL,Norm,GasA,Pave,Y,PConc,TA,TA,Gd,TA,No,Unf,TA,TA,60,6,5,1999,2000,0.0,0,0,953,953,694,0,1647,0,0,2,1,1,7,1999.0,460,40,0,0,0,8,2007,1456,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,No,ALQ,TA,TA,20,6,6,1978,1988,119.0,790,163,589,2073,0,0,2073,1,0,2,0,1,7,1978.0,500,0,0,0,0,2,2010,1457,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,Rec,TA,SBrkr,Min1,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,RL,Norm,GasA,Pave,Y,Stone,Ex,Gd,TA,Gd,No,GLQ,Gd,Gd,70,7,9,1941,2006,0.0,275,0,877,1188,1152,0,2340,0,0,2,0,1,9,1941.0,252,60,0,0,2500,5,2010,1458,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,TA,TA,Mn,GLQ,Gd,NaN,20,5,6,1950,1996,0.0,49,1029,0,1078,0,0,1078,1,0,1,0,1,5,1950.0,240,0,112,0,0,4,2010,1459,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,1Fam,1Story,

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
# X and y creation
y = df.pop("Expensive")

# Feature Engineering
X = df.drop(columns=["LotFrontage"])

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# building the pipeline
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

ordinal_vars = ['LandContour', 'LandSlope',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC']
non_ordinal_vars = ['MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 
                                         'Foundation', 'Alley', 'LotShape', 'Utilities', 'LotConfig', 
                                         'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 
                                         'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                         'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature',
                                         'SaleType', 'SaleCondition']
LandContour = ['Lvl', 'Bnk', 'HLS', 'Low']
LandSlope = ['Gtl', 'Mod', 'Sev']
ExterQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
ExterCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
BsmtQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtExposure = ['Gd', 'Av', 'Mn', 'No', 'N_A']
BsmtFinType1 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
BsmtFinType2 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
HeatingQC = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
KitchenQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
Functional = ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']
FireplaceQu = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageFinish = ['Fin', 'RFn', 'Unf', 'N_A']
GarageQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
PoolQC = ['Ex', 'Gd', 'TA', 'Fa', 'N_A']


ordinal_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OrdinalEncoder(categories=[LandContour,LandSlope,
                                                    ExterQual,ExterCond, BsmtQual, BsmtCond,
                                                    BsmtExposure, BsmtFinType1, BsmtFinType2,
                                                    HeatingQC, KitchenQual, Functional,
                                                    FireplaceQu, GarageFinish, GarageQual, GarageCond,
                                                    PoolQC])
)

non_ordinal_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore")
)

cat_transformers = [
    ("ordinal_pipe", ordinal_pipe, ordinal_vars),
    ("non_ordinal_pipe", non_ordinal_pipe, non_ordinal_vars)
]

numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)
"""
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    #OneHotEncoder(handle_unknown="ignore")
    OrdinalEncoder(categories='auto')
)
"""

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        *cat_transformers
    ]
)

In [9]:
#normalizing/scaling

#choose columns -> numeric & > 1
df
#df_n = MinMaxScaler().fit_transform

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,KitchenQual,FireplaceQu,MSSubClass,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,BsmtFinType2,HeatingQC,Electrical,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,No,GLQ,Gd,NaN,60,7,5,2003,2003,196.0,706,0,150,856,854,0,1710,1,0,2,1,1,8,2003.0,548,61,0,0,0,2,2008,1,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,RL,Feedr,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,Gd,ALQ,TA,TA,20,6,8,1976,1976,0.0,978,0,284,1262,0,0,1262,0,1,2,0,1,6,1976.0,460,0,0,0,0,5,2007,2,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,Mn,GLQ,Gd,TA,60,7,5,2001,2002,162.0,486,0,434,920,866,0,1786,1,0,2,1,1,6,2001.0,608,42,0,0,0,9,2008,3,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,RL,Norm,GasA,Pave,Y,BrkTil,TA,TA,TA,Gd,No,ALQ,Gd,Gd,70,7,5,1915,1970,0.0,216,0,540,961,756,0,1717,1,0,1,0,1,7,1998.0,642,35,272,0,0,2,2006,4,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,NaN,Unf,Gd,SBrkr,Typ,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,Av,GLQ,Gd,TA,60,8,5,2000,2000,350.0,655,0,490,1145,1053,0,2198,1,0,2,1,1,9,2000.0,836,84,0,0,0,12,2008,5,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,RL,Norm,GasA,Pave,Y,PConc,TA,TA,Gd,TA,No,Unf,TA,TA,60,6,5,1999,2000,0.0,0,0,953,953,694,0,1647,0,0,2,1,1,7,1999.0,460,40,0,0,0,8,2007,1456,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,No,ALQ,TA,TA,20,6,6,1978,1988,119.0,790,163,589,2073,0,0,2073,1,0,2,0,1,7,1978.0,500,0,0,0,0,2,2010,1457,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,Rec,TA,SBrkr,Min1,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,RL,Norm,GasA,Pave,Y,Stone,Ex,Gd,TA,Gd,No,GLQ,Gd,Gd,70,7,9,1941,2006,0.0,275,0,877,1188,1152,0,2340,0,0,2,0,1,9,1941.0,252,60,0,0,2500,5,2010,1458,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,NaN,Unf,Ex,SBrkr,Typ,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,TA,TA,Mn,GLQ,Gd,NaN,20,5,6,1950,1996,0.0,49,1029,0,1078,0,0,1078,1,0,1,0,1,5,1950.0,240,0,112,0,0,4,2010,1459,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,NaN,

# Modelling Pipe - 1

## Decisiontree - Creation

In [112]:
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}

'''param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    'decisiontreeclassifier__max_depth': range(2, 20),
    'decisiontreeclassifier__min_samples_leaf': range(3, 15, 2),
    'decisiontreeclassifier__min_samples_split': range(3, 40, 5),
    'decisiontreeclassifier__criterion':['gini', 'entropy']
    }'''

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"dtree" : search.best_score_}

scores

Fitting 5 folds for each of 60 candidates, totalling 300 fits


ValueError: 
All the 300 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramData\anaconda3\envs\test\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\ProgramData\anaconda3\envs\test\lib\site-packages\sklearn\pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\ProgramData\anaconda3\envs\test\lib\site-packages\sklearn\pipeline.py", line 370, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Pro

In [ ]:
sub_df = pd.read_csv(r'ids.csv')

X_sumbmition = pd.read_csv(r'test-housing-classification.csv')
sub_df['Expensive'] = search.predict(X_sumbmition) # only cange the piplene! 
sub_df.to_csv(r'submission_x.csv', index=False) #only cahnge ist 

In [ ]:
sub_df

## Decisiontree - Analysis DT

### Confusion Matrix

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

Training Accuracy: 0.946917808219178
Testing Accuracy: 0.934931506849315
Confusion Matrix (Training):
 [[968  25]
 [ 37 138]]
Confusion Matrix (Testing):
 [[243   7]
 [ 12  30]]


### F-Score

In [43]:
from sklearn.metrics import f1_score
print("Training Accuracy:", f1_score(y_train, y_train_pred)) # overfitting, less leafev 
print("Testing Accuracy:", f1_score(y_test, y_test_pred))

Training Accuracy: 0.8165680473372781
Testing Accuracy: 0.759493670886076


### Kappa

In [44]:
from sklearn.metrics import cohen_kappa_score
print("Training Accuracy:", cohen_kappa_score(y_train, y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test, y_test_pred))

Training Accuracy: 0.7855827367099552
Testing Accuracy: 0.7220440881763528


# Modeling Pipe - 2


In [ ]:
# Modeling Pipe - 2

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


full_pipeline = make_pipeline(preprocessor,
                              KNeighborsClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors": range(3, 10, 2)
}

search2 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search2.fit(X_train, y_train)

scores2 = {"knn" : search.best_score_}

scores2


## Decisiontree - Analysis KNN

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Get predictions for the training and testing datasets
y_train_pred = search2.predict(X_train)
y_test_pred = search2.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

### F-Score

In [ ]:
from sklearn.metrics import f1_score
print("Training Accuracy:", f1_score(y_train,y_train_pred))
print("Testing Accuracy:", f1_score(y_test,y_test_pred))

### Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score
print("Training Accuracy:", cohen_kappa_score(y_train,y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test,y_test_pred))

In [ ]:
sub_df = pd.read_csv(r'ids.csv')

X_sumbmition = pd.read_csv(r'test-housing-classification.csv')
sub_df['Expensive'] = search.predict(X_sumbmition) # only cange the piplene! 
sub_df.to_csv(r'submission_2.csv', index=False) #only cahnge ist 

## Support Vector Machine - Creation

In [96]:
# Modelling Pipe - 3

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Assuming you have already defined 'preprocessor', 'X_train', and 'y_train'

#full_pipeline = make_pipeline(preprocessor, SVC())
svm_model = SVC(C=1.0, kernel='rbf')
full_pipeline = make_pipeline(preprocessor, svm_model)
param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["mean", "median"],
    'svc__gamma': [0.00001, 0.0001, 0.001],
    'svc__C': [0.001,0.01,0.1],
    'svc__kernel': ['linear']}

search_svc = GridSearchCV(full_pipeline, param_grid, cv=3, verbose=1)
search_svc.fit(X_train, y_train)

scores_svc = {"svc": search_svc.best_score_}
print("Best parameters: ", search_svc.best_params_)
print(scores_svc)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'mean', 'svc__C': 0.1, 'svc__gamma': 1e-05, 'svc__kernel': 'linear'}
{'svc': 0.943497462263529}


In [97]:
# Get predictions for the training and testing datasets
y_train_pred = search_svc.predict(X_train)
y_test_pred = search_svc.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

Training Accuracy: 0.9708904109589042
Testing Accuracy: 0.9554794520547946
Confusion Matrix (Training):
 [[981  12]
 [ 22 153]]
Confusion Matrix (Testing):
 [[246   4]
 [  9  33]]


In [98]:
#F-Score
print('##F-SCORE##')
print("Training Accuracy:", f1_score(y_train,y_train_pred))
print("Testing Accuracy:", f1_score(y_test,y_test_pred))
#Kappa
print('##KAPPA-SCORE##')
print("Training Accuracy:", cohen_kappa_score(y_train,y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test,y_test_pred))

##F-SCORE##
Training Accuracy: 0.9
Testing Accuracy: 0.8354430379746834
##KAPPA-SCORE##
Training Accuracy: 0.882983174706073
Testing Accuracy: 0.8098196392785572


In [92]:
sub_df = pd.read_csv(r'ids.csv')

X_sumbmition = pd.read_csv(r'test-housing-classification.csv')
sub_df['Expensive'] = search_svc.predict(X_sumbmition) # only cange the piplene! 
sub_df.to_csv(r'submission_7.csv', index=False) #only cahnge ist 

# Pipe3 - RandomForest

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["median"],
    "randomforestclassifier__n_estimators": [120],
    "randomforestclassifier__max_depth": [10],
    "randomforestclassifier__min_samples_split": [2],
    "randomforestclassifier__min_samples_leaf": [1]
}

search4 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search4.fit(X_train, y_train)

scores4 = {"rf" : search4.best_score_}
print("Best parameters: ", search4.best_params_)
scores4

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 120}


{'rf': 0.93920252375188}

In [83]:

# Get predictions for the training and testing datasets
y_train_pred = search4.predict(X_train)
y_test_pred = search4.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", accuracy_test)

print("Confusion Matrix (Training):\n", confusion_matrix_train)
print("Confusion Matrix (Testing):\n", confusion_matrix_test)

Training Accuracy: 0.9991438356164384
Testing Accuracy: 0.9691780821917808
Confusion Matrix (Training):
 [[993   0]
 [  1 174]]
Confusion Matrix (Testing):
 [[248   2]
 [  7  35]]


In [84]:
#F-Score
print('##F-SCORE##')
print("Training Accuracy:", f1_score(y_train,y_train_pred))
print("Testing Accuracy:", f1_score(y_test,y_test_pred))
#Kappa
print('##KAPPA-SCORE##')
print("Training Accuracy:", cohen_kappa_score(y_train,y_train_pred))
print("Testing Accuracy:", cohen_kappa_score(y_test,y_test_pred))

##F-SCORE##
Training Accuracy: 0.9971346704871061
Testing Accuracy: 0.8860759493670887
##KAPPA-SCORE##
Training Accuracy: 0.9966314040815385
Testing Accuracy: 0.8683366733466934


In [85]:
sub_df = pd.read_csv(r'ids.csv')

X_sumbmition = pd.read_csv(r'test-housing-classification.csv')
sub_df['Expensive'] = search4.predict(X_sumbmition) # only cange the piplene! 
sub_df.to_csv(r'submission_5.csv', index=False) #only cahnge ist 